In [ ]:
!pip install pytorch-lightning==1.2.3 --quiet

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torchvision.utils import make_grid

import pytorch_lightning as pl

In [ ]:
pl.__version__

In [ ]:
# Use kaggle username and token to download the dataset
dataset_url = 'https://www.kaggle.com/tongpython/cat-and-dog'
od.download(dataset_url)

In [ ]:
!ls cat-and-dog/test_set/test_set

In [ ]:
image_size = 64
batch_size = 256

data_path_train = "cat-and-dog/training_set/training_set"
data_path_test= "cat-and-dog/test_set/test_set"

In [ ]:
train_dataset = ImageFolder(data_path_train, transform=T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor()]))

test_dataset = ImageFolder(data_path_test, transform=T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor()]))

train_dataloader = DataLoader(train_dataset, batch_size, num_workers=2, pin_memory=True, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, num_workers=2, pin_memory=True)

In [ ]:
print("Total images in train dataset:", len(train_dataset))
print("Tottal images in test dataset:", len(test_dataset))

In [ ]:
def display_images(images, total_images_to_display = 64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([])
    ax.set_yticks([])
    images= images.detach()
    images = images[:total_images_to_display]

    ax.imshow(make_grid(images, nrow=8).permute(1, 2, 0))

In [ ]:
for images, _ in train_dataloader:
  display_images(images)
  break

In [ ]:
for images, _ in test_dataloader:
  display_images(images)
  break

In [ ]:
class ImageClassifier(pl.LightningModule):

    def __init__(self, learning_rate = 0.001):
        super().__init__()

        self.learning_rate = learning_rate

        self.conv_layer1 = nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        self.conv_layer2 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        self.fully_connected_1 =nn.Linear(in_features=32 * 32 * 6,out_features=1000)
        self.fully_connected_2 =nn.Linear(in_features=1000,out_features=250)
        self.fully_connected_3 =nn.Linear(in_features=250,out_features=60)
        self.fully_connected_4 =nn.Linear(in_features=60,out_features=2)
        self.loss = nn.CrossEntropyLoss()
        self.save_hyperparameters()

    def forward(self, input):
        output=self.conv_layer1(input)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv_layer2(output)
        output=self.relu2(output)
        output=output.view(-1, 6*32*32)
        output = self.fully_connected_1(output)
        output = self.fully_connected_2(output)
        output = self.fully_connected_3(output)
        output = self.fully_connected_4(output)
        return output

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs) 
        accuracy = self.binary_accuracy(outputs, targets)
        loss = self.loss(outputs, targets)
        self.log('train_accuracy', accuracy, prog_bar=True)
        self.log('train_loss', loss)
        return {"loss":loss, "train_accuracy":accuracy}

    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        accuracy = self.binary_accuracy(outputs,targets)
        loss = self.loss(outputs, targets)
        self.log('test_accuracy', accuracy)
        return {"test_loss":loss, "test_accuracy":accuracy}

    def configure_optimizers(self):
        params = self.parameters()
        optimizer = optim.Adam(params=params, lr = self.learning_rate)
        return optimizer

    def binary_accuracy(self, outputs, targets):
        _, outputs = torch.max(outputs,1)
        correct_results_sum = (outputs == targets).sum().float()
        acc = correct_results_sum/targets.shape[0]
        return acc

In [ ]:
model = ImageClassifier()

trainer = pl.Trainer(max_epochs=100, progress_bar_refresh_rate=30, gpus=1)
trainer.fit(model, train_dataloader=train_dataloader)

In [ ]:
trainer.test(test_dataloaders=test_dataloader)